In [ ]:
import pandas as pd
import math
import re
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
df=pd.read_csv("../input/bengaluru-house-price-data/Bengaluru_House_Data.csv")

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
ls=list()
ls=round(100*(df.isnull().sum()/len(df.index)),4)
ls

****Now from above list we can conclude that society has 41% null values and Their is no sense to take that feature****

In [ ]:
df=df.drop("society",axis=1)

In [ ]:
df["location"].value_counts()

In [ ]:
df["location"].fillna("Whitefield",inplace=True)

In [ ]:
df['size'].value_counts()

In [ ]:
df["size"].fillna("2 BHK",inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df["bath"].fillna(2.692610,inplace=True)

In [ ]:
df["balcony"].fillna(1.584376,inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df['size'] = df['size'].str[0:2]
df['size']=df['size'].astype('int')
df

In [ ]:
df.info()


****Now we can see that total_sqrt is of object datatype we have to convert it into int but their is some range in the column so we have to deal with it also and map the total_sqrt with area_type****

In [ ]:
df.total_sqft.unique()

In [ ]:
def convert_total_sqft(my_list):
    if len(my_list) == 1:
        
        try:
            return float(my_list[0])
        except:
            ls = ['Sq. Meter', 'Sq. Yards', 'Perch', 'Acres', 'Cents', 'Guntha', 'Grounds']
            split_list = re.split('(\d*.*\d)', my_list[0])[1:]
            area = float(split_list[0])
            type_of_area = split_list[1]
            if type_of_area == 'Sq. Meter':
                area_in_sqft = area * 10.7639
            elif type_of_area == 'Sq. Yards':
                area_in_sqft = area * 9.0
            elif type_of_area == 'Perch':
                area_in_sqft = area * 272.25
            elif type_of_area == 'Acres':
                area_in_sqft = area * 43560.0
            elif type_of_area == 'Cents':
                area_in_sqft = area * 435.61545
            elif type_of_area == 'Guntha':
                area_in_sqft = area * 1089.0
            elif type_of_area == 'Grounds':
                area_in_sqft = area * 2400.0
            return float(area_in_sqft)
        
    else:
        return (float(my_list[0]) + float(my_list[1]))/2.0

In [ ]:
df['total_sqft'] = df.total_sqft.str.split('-').apply(convert_total_sqft)

****When we process the bath and size attribute we came across some conclusion that size of a flat can't be greator than 5 bhk or washroom's can't be greater than 6****

In [ ]:
df = df.drop(df[df['bath']>5].index)
df = df.drop(df[df['size']>6].index)

In [ ]:
df['price_per_sqft'] = df['price']*100000/df['total_sqft']

In [ ]:
df

****Now we have earlier study in our 12th class that if we have a couple of data and if the value if greator than sum of mean and deviation or less than the difference between mean and standard deviation then the data point is invalid , in our case it's a outlier so we have to remove that Outlier****

In [ ]:
def remove_outlier(df1):
    new_dataframe = pd.DataFrame()
    for key, df2 in df1.groupby('location'):
        m = np.mean(df2.price_per_sqft)
        st = np.std(df2.price_per_sqft)
        reduced_df = df2[(df2.price_per_sqft>(m-st)) & (df2.price_per_sqft<=(m+st))]
        new_dataframe = pd.concat([new_dataframe,reduced_df],ignore_index=True)
    return new_dataframe
    

In [ ]:
df = remove_outlier(df)
df.shape

In [ ]:
df = remove_outlier(df)

In [ ]:
df.drop(columns=['availability','area_type'],inplace = True)

In [ ]:
df.location = df.location.str.strip()
location_count = df['location'].value_counts(ascending=False)

In [ ]:
location_stats_less_than_8 = location_count[location_count<=8]

In [ ]:
df.location = df.location.apply(lambda x: 'other' if x in location_stats_less_than_8 else x)

In [ ]:
df = df[df.location != 'other']

In [ ]:
df.shape

In [ ]:
location = pd.get_dummies(df.location)

In [ ]:
df = pd.concat([df,location],axis='columns')

In [ ]:
df.shape

In [ ]:
df1 = df.drop('location',axis = 1)
df1 = df1.drop(columns=['balcony','price_per_sqft'])
df1

In [ ]:
x=df1.drop("price",axis=1)
y=df1["price"]

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.4)
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.transform(xtest)

In [ ]:
xtrain

In [ ]:
!pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
models,predictions = reg.fit(xtrain, xtest, ytrain, ytest)

In [ ]:
models

In [ ]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=1, max_iter=1000).fit(xtrain, ytrain)
print("Train Accuracy ",regr.score(xtrain,ytrain)*100)
print("Test Accuracy ",regr.score(xtest, ytest)*100)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
model_params = {
    'MLP': {
        'model': MLPRegressor(),
        'params' : {
            'max_iter':[1000],
            'random_state':[1]
        }  
    },
    'random_forest': {
        'model': RandomForestRegressor(),
        'params' : {
            'n_estimators': [1,5,10,20,25]
        }
    },
    'GradientBoosting':{
        'model':GradientBoostingRegressor(random_state=0),
        'params':{
            'n_estimators':[1500]
        }
    }
}

In [ ]:
scores=[]
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(xtrain,ytrain)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [ ]:
df2 = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df2